In [1]:
from training.perceiver import*
from training.utils import*
from training.losses import*
from training.VIT import*
from training.ResNet import*
from collections import defaultdict
from training import*

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

import torch
import numpy as np
from torch import nn, einsum
import torch.nn.functional as F
import einops as einops
from einops import rearrange, repeat
from einops.layers.torch import Reduce
from pytorch_lightning.profilers import AdvancedProfiler
import matplotlib.pyplot as plt

from configilm import util
util.MESSAGE_LEVEL = util.MessageLevel.INFO  # use INFO to see all messages


from configilm.extra.DataSets import BENv2_DataSet
from configilm.extra.DataModules import BENv2_DataModule

torch.manual_seed(0)

In [2]:
mode="validation"

In [3]:
def prepare_BENv2(mode=None,max_len=None):
    dico_paths={"images_lmdb":"data/Encoded-BigEarthNet/",
    "metadata_parquet":"data/Encoded-BigEarthNet/metadata.parquet",
    "metadata_snow_cloud_parquet":"data/Encoded-BigEarthNet/metadata_for_patches_with_snow_cloud_or_shadow.parquet"}

    df=open_parquet(dico_paths["metadata_parquet"])

    if max_len!=None:
        return BENv2_DataSet.BENv2DataSet(data_dirs=dico_paths, img_size=(14, 120, 120),split=mode,max_len=max_len),df

    
    return BENv2_DataSet.BENv2DataSet(data_dirs=dico_paths, img_size=(14, 120, 120),split=mode),df



In [4]:
def create_datasets(name,trans_conf,trans_tokens,sizes=(2,2,2),max_len=None,max_len_h5=-1):

   
    mode="train"
    ds,df=prepare_BENv2(max_len=max_len)

    idxs,_=get_tiny_dataset(ds,df,MAX_IDs=sizes[0],mode=mode)
    #idxs=None
    stats=create_dataset(idxs, ds,df, name=name, mode=mode, trans_config=trans_conf,trans_tokens=trans_tokens,stats=None,max_len=max_len_h5)

    mode="validation"
    ds,df=prepare_BENv2(max_len=max_len)
    
    
    idxs,_=get_tiny_dataset(ds,df,MAX_IDs=sizes[1],mode=mode)
    #idxs=None
    create_dataset(idxs, ds,df, name=name, mode=mode,trans_config=trans_conf,trans_tokens=trans_tokens,stats=stats,max_len=max_len_h5)

    mode="test"
    ds,_=prepare_BENv2(max_len=max_len)
    idxs,_=get_tiny_dataset(ds,df,MAX_IDs=sizes[2],mode=mode)
    #idxs=None
    create_dataset(idxs, ds,df, name=name, mode=mode,trans_config=trans_conf,trans_tokens=trans_tokens,stats=stats,max_len=max_len_h5)

    

In [5]:
bands_yaml="./data/bands_info/bands.yaml"
configs_dataset="./data/Tiny_BigEarthNet/configs_dataset_tiny.yaml"
config_dico = read_yaml("./training/configs/config_test-Atomiser_Atos.yaml")
#test_conf= transformations_config(config_dico,bands_yaml,configs_dataset,path_imgs_config="./data/Tiny_BigEarthNet/",name_config="BigEarthPart")
#(self,configs_dataset,path_imgs_config,name_config=""):
modalities_trans= modalities_transformations_config(configs_dataset,name_config="tiny")
test_conf= transformations_config(bands_yaml,config_dico)

#create_datasets("tiny",modalities_trans,trans_tokens=test_conf,sizes=(1000,1000,1000),max_len_h5=-1)

In [6]:


data_module=Tiny_BigEarthNetDataModule( "./data/Tiny_BigEarthNet/tiny", 
                                       batch_size=16, 
                                       num_workers=4,
                                       trans_modalities=modalities_trans,
                                       trans_tokens=None,
                                       model="Atomiser")

data_module.setup()
# Prepare dataloaders
train_loader = data_module.train_dataloader()
#val_loader = data_module.val_dataloader()
#test_loader = data_module.test_dataloader()

Train DataLoader created on rank: 0


In [8]:
xp_name="test_xp"
config_model = "Atomiser_Atos"
config_name_dataset = "tiny"
config_name_dataset= "./data/custom_flair/"+config_name_dataset
from pytorch_lightning import Trainer,seed_everything
from pytorch_lightning.callbacks import EarlyStopping

seed_everything(42, workers=True)

torch.set_default_dtype(torch.float32)
torch.set_float32_matmul_precision('medium')

config_model = read_yaml("./training/configs/config_test-"+config_model+".yaml")
#labels=load_json_to_dict("./data/flair_2_toy_dataset/flair_labels.json")
bands_yaml = "./data/Tiny_BigEarthNet/bands.yaml"

bands_yaml="./data/bands_info/bands.yaml"
configs_dataset="./data/Tiny_BigEarthNet/configs_dataset_tiny.yaml"
config_dico = read_yaml("./training/configs/config_test-Atomiser_Atos.yaml")

modalities_trans= modalities_transformations_config(configs_dataset,name_config="tiny")
test_conf= transformations_config(bands_yaml,config_dico)

data_module=Tiny_BigEarthNetDataModule( "./data/Tiny_BigEarthNet/tiny", 
                                       batch_size=16, 
                                       num_workers=4,
                                       trans_modalities=modalities_trans,
                                       trans_tokens=None,
                                       model="Atomiser")
data_module.setup()
# Prepare dataloaders

wand = False
wandb_logger = None
if wand:
    if os.environ.get("LOCAL_RANK", "0") == "0":
        import wandb
        wandb.init(
            name=config_model['encoder'],
            project=config_name_dataset+"_modalities",
            config=config_model
        )
        wandb_logger = WandbLogger(project=config_name_dataset+"_modalities")

#def __init__(self, config, wand, name)
model = Model(config_model,wand=wand, name=xp_name,transform=test_conf)


early_stop_callback = EarlyStopping(monitor="val_ap", min_delta=0.00, patience=15, verbose=False, mode="max")

profiler = AdvancedProfiler(dirpath="profiling", filename="profiler_output.txt")

# Configure the trainer for distributed training.
trainer = Trainer(
    use_distributed_sampler=False,  # we use our custom sampler
    #strategy="ddp",
    max_epochs=config_model["trainer"]["epochs"],
    logger=wandb_logger,
    log_every_n_steps=1,
    #devices=1,
    accelerator="gpu",
    callbacks=[early_stop_callback],
    profiler=profiler  
)

trainer.fit(model, datamodule=data_module)



Seed set to 42
INFO: You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:lightning.pytorch.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO: GPU available: True (mps), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (mps), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: Loading `train_dataloader` to estimate number of stepping batches.
INFO:lightning.pytorch.utilities.rank_zero:Loading `train_dataloader` to estimate number of stepping batches.


Train DataLoader created on rank: 0


/Users/hugoriffaud/Documents/Atomiser_BigEarthNet/venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)

   | Name                                    | Type                       | Params | Mode 
------------------------------------------------------------------------------------------------
0  | transform                               | transformations_config     | 0      | train
1  | metric_train_AP_per_class               | MultilabelAveragePrecision | 0      | train
2  | metric_train_accuracy_per_class         | MultilabelAccuracy         | 0      | train
3  | metric_val_mod_val_AP_per_class         | MultilabelAveragePrecision | 0      | train
4  | metric_val_mod_val_accuracy_per_class   | MultilabelAccuracy         | 0      | train
5  | metric_val_mod_train_AP_per_class       | MultilabelAveragePrecision | 0      | trai

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Validation DataLoader created on rank: 0
torch.Size([16, 12, 120, 120])


RuntimeError: linear(): input and weight.T shapes cannot be multiplied (16x1000 and 768x19)